In [0]:
!pip install pysoundfile

In [0]:
# Importing Libraries
import numpy as np
import soundfile as sf
import scipy.fftpack as spfft
import cvxpy as cvx
import math
from IPython.display import Audio
from sklearn.linear_model import OrthogonalMatchingPursuit


In [0]:
from google.colab import files
uploaded = files.upload()

Saving sum_sig.wav to sum_sig (7).wav
Saving test9.wav to test9 (4).wav


In [0]:
sum_sig, rate = sf.read('sum_sig.wav')                # Reading the training set "sum_sig" which consists of all the training signals summed together

n = len(sum_sig)                                     # Length of sum_sig
m = int(n*(3/10))                                    # Taking 30% of the total samples for reconstruction

sum_sigf = spfft.dct(sum_sig, norm='ortho')          # Finding DCT of the sum_sig to figure out maximum energy points

ind=np.flip(np.argsort(sum_sigf),-1)                 # Finding out the indices of the points containing maximum energy

ind = ind[:m]                                        # Taking first 30% samples of the maximum points

sensing_matrix = np.zeros((m,n))                     # Creating a zero matrix of size m by n in order to design the required sensing matrix

for i in range(m):                                  # Inserting 1's in the indices containing maximum energy in order to design the required matrix
    sensing_matrix[i,ind[i]] = 1

In [0]:
test_sig, rate = sf.read('test9.wav')               # Reading the test signal for reconstruction

sub_samp = np.dot(sensing_matrix, test_sig)         # Taking samples according to the designed sensing matrix

A = spfft.idct(np.identity(n), norm='ortho', axis=0)       # create idct matrix operator

A1 = np.dot(sensing_matrix,A)             

In [0]:
# do L1 optimization
%%time
vx = cvx.Variable(n)
objective = cvx.Minimize(cvx.norm(vx, 1))
constraints = [A1*vx == sub_samp]
prob = cvx.Problem(objective, constraints)
result = prob.solve(verbose=True)
x = np.array(vx.value)
x = np.squeeze(x)
cvx_sig = spfft.idct(x, norm='ortho', axis=0)

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 4162, constraints m = 4786
          nnz(P) + nnz(A) = 1306868
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -1.6648e+04   8.00e+00   2.66e+04   1.00e-01   1.79e+00s
 200   4.0974e+01   1.93e-02   1.63e-03   1.00e-01   3.19e+00s
 400   4.3269e+01   5.42e-03   9.59e-04   1.00e-01   4.57

In [0]:
# L1 SNR and Sound
CVX_SNR = 10*math.log10(np.sum(cvx_sig**2)/np.sum((cvx_sig-test_sig)**2))
print(CVX_SNR)

Audio(data=cvx_sig, rate=rate)

5.641281499252965


In [0]:
# Orthogonal Matching Pursuit
%%time
n_nonzero_coefs = m
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
omp.fit(A1, sub_samp)
coef = omp.coef_
omp_sigt = spfft.idct(coef, norm='ortho', axis=0)

CPU times: user 1.53 s, sys: 647 ms, total: 2.18 s
Wall time: 1.12 s


In [0]:
# OMP SNR and Sound
O_SNR = 10*math.log10(np.sum(omp_sigt**2)/np.sum((omp_sigt-test_sig)**2))
print(O_SNR)

Audio(data=omp_sigt, rate=rate)

0.8978460042085579


In [0]:
# Linear Reconstruction
%%time
linear_sig = spfft.idct(np.dot(np.transpose(A1),sub_samp), norm='ortho', axis=0)

CPU times: user 10.1 ms, sys: 10.7 ms, total: 20.8 ms
Wall time: 18.6 ms


In [0]:
# Linear SNR and Sound
L_SNR = 10*math.log10(np.sum(linear_sig**2)/np.sum((linear_sig-test_sig)**2))
print(L_SNR)

Audio(data=linear_sig, rate=rate)

-5.134513325301482
